# Connecting Streamlit with TF - TF Fullstack

### Introduction

So in the previous lesson, we saw how we can use terraform to set up our infrastructure and set up our flask api using docker.  In this lesson, we can move onto connecting our streamlit application.

### Building and connecting our frontend

* Dockerizing our frontend streamlit app

Ok, so the first step is to dockerize our streamlit application.  If you navigate to the `frontend` folder, and then the Dockerfile, you'll see the following:

```Dockerfile
CMD ["streamlit", "run", "./index.py", "--server.address=0.0.0.0", "--server.port=80"]
```

Ok, so here we have our normal command to start up our streamlit app, but then we pass some additional arguments.  The `--server.port` argument, tells streamlit to run on port 80.  And remember, we'll want that port because this is the dashboard a web browser will connect to - and web browsers by default make requests to port 80.  We also specify a server address as port `0.0.0.0`, which will tell streamlit to accept requests from external environments (eg. computerS), just like it does for flask.

* Ok, so let's build our streamlit app -- you can build it, specifying the dockerhub username you'll ultimately want to deploy it to.  We'll test it out locally, so you don't need the platform argument just yet.

Once you build the streamlit frontend, we'll want to also bootup the backend application (on our laptop), so that we can see how the streamlit frontend can make requests to the backend api.  

Before moving on, let's see explore how this can occur.  If you at the `frontend/app/settings.py` file, you'll see the following.

```python
API_HOST = os.environ.get('API_HOST')
API_PORT = os.environ.get('API_PORT')
api_url = f"http://{API_HOST}:{API_PORT}/positions"
```

So our `api_url` defines where the streamlit app will make requests to.  And in the `.env` file you'll see the host defined as `backend` and the port as 5000.  So putting this together with the `view_functions.py` code, you can see that our streamlit app makes a request to `http://backend:5000/positions`

```python
from settings import api_url

def find_positions():
    response = requests.get(api_url)
    ...
```

How do we make a request to `http://backend:5000`?  Well, we can do this, so long as we also have booted up our image of our backend api, and given it a container name of `backend`.  We'll also need to create a docker network, and make sure they both run on the same network.

### Connecting our containers with a network

So let's move onto do this.  We can create a docker network, where each of our containers will live on with:

```bash
docker network create jobs-scraper
```

And from there, we can tell the backend container to run on that network with the following.

```bash
docker run -d -p 5000:5000 --network jobs-scraper --name backend -e DB_USERNAME=$DB_USERNAME -e DB_PASSWORD=$DB_PASSWORD -e DB_NAME=$DB_NAME -e DB_HOST=$DB_HOST $BACKEND_IMAGE
```

> You can export the needed variables (look for what you specified in your `ec2-setup.sh` file previously)

And do the same for the frontend application.

> Change `backend_image_name` to match your image's name.

```bash
docker run -d -p 80:80 --network jobs-scraper --name frontend backend_image_name
```

> The `-d` means that each container will be run as a daemon, that is, a background process.  You can still confirm both are running with `docker ps`, and shut them down with `docker stop container_name` and `docker rm container_name`.

### Encoding it

Ok, so now we should update our `ec2-setup.sh` file, confirm that our code works on our laptop, and then we can update our `ec2-setup.tpl` file.

So in the `ec2-setup.sh` file, do the following:

1. Write code to pull the `FRONTEND_IMAGE`.  Include a `while` block that will repull the image if it's not there, and then sleep for five seconds.

2. Add code to stop and remove the frontend container, but have an `if` block to only run the commands if the container is already running.

3. Add code to create the network, but only if the network does not exist.  If the network already exists, use `echo` to indicate you are skipping this step.

4. Then add the code to run both the containers on the network

* Confirm that the app properly boots up.

```
sh ec2-setup.sh
```

* Before moving on, do some refactoring by assigning and properly referencing variables of `NETWORK_NAME`, `FRONTEND_CONTAINER`, and `FRONTEND_IMAGE`.

Towards the top of the file, you should have the following variables declared:

```bash
export NETWORK_NAME=""
export BACKEND_CONTAINER=""
export FRONTEND_CONTAINER=""
export BACKEND_IMAGE=""
export FRONTEND_IMAGE=""
export DB_USERNAME=""
export DB_PASSWORD=""
export DB_NAME=""
```

### Moving to AWS

Ok, so now let's begin to update this code to work on our AWS instance.  

* Rebuild the image
    * To start, we'll need to rebuild the image specifying the platform of `--platform=linux/amd64/v2`, and when tagging the image specify a different version (eg. `jek2141/scraper_frontend:amd_v2`).  
    * Push up the image when it's ready


* Update the terraform script and template to add our new procedures
    * We'll let you do this.  Remember to add any `sudo` prefixes to your docker commands).  
    > You can check your work with `terraform console`

After deploying you can check your work by shing into the ec2 machine.  You can check your work by confirming that both containers are up and running.  And if there are errors remember you can see the logs with:
    
```bash
cat /var/log/cloud-init-output.log
cat /var/log/cloud-init.log
```

If things look good you can go to your ec2 domain name, and see if you get your dashboard when making an `http://` request.

<img src="./complete-deployment.png" width="60%">

Exhale.  If this isn't nice, what is.

### Summary

In this lesson, we placed the finishing touches on deploying our application.  The main step was to make use of a docker network.  Remember, this allowed us to make a request to something like:

```
http://backend:5000/positions
``` 

From our frontend dashboard.  And this worked so long as we named our backend container `backend`, and also created a network, booted up both the frontend and backend containers on those networks.  And we do this by first creating a network, and then specifying that network when running the container.

```
docker network create jobs-scraper

docker run -d -p 80:80 --network jobs-scraper --name frontend backend_image_name
```

### Resources

[Terraform templates environmental issue](https://discuss.hashicorp.com/t/template-v2-2-0-does-not-have-a-package-available-mac-m1/35099/3)